#### This notebook contains code used to populate data dictionary
It is done step by step with different chunks of calls.

It is in development state and not cleaned at all!

In [1]:
import pandas as pd
from data_dictionary.append_df_to_excel import append_df_to_excel

In [2]:
# read one sheet from excel tmee database provided by Eduard
path_read_file = './populate_data_dictionary/TM Indicators Flow Revised Database - 2020306 SP ED ECD -working file v-6 Aug 2020_beto.xlsx'
indicators = pd.read_excel(path_read_file, sheet_name='TM Revised database', header = 2)

#### Indicators New or Retained

In [3]:
# filter indicators that are new or retained from excel tmee data base
logic_not_removed = indicators['Status in the new `db'].str.lower().str.contains('new|retained')
# filter logic_not_removed that are not boolean
logic_not_null = logic_not_removed.notnull()
# operator AND for two logics above
logic_not_removed_null = logic_not_removed & logic_not_null
not_removed_indicators = indicators[logic_not_removed_null]

#### Indicators specified with Unesco Institute for Statistics (UIS) API source address

In [4]:
# point to indicators that are extracted from UIS
logic_UIS = not_removed_indicators['Data Source'].str.lower().str.contains("api.uis.unesco.org")
# filter not boolean from logic_UIS
logic_UIS_not_null = logic_UIS.notnull()
# operator AND for two logics above
logic_UIS_not_null = logic_UIS & logic_UIS_not_null
print(f"There are {logic_UIS_not_null.sum()} indicators extracted from UIS.")

There are 129 indicators extracted from UIS.


In [5]:
# create new column in dataframe with their corresponding codes
not_removed_indicators['Code'] = not_removed_indicators['Indicator Code'][logic_UIS_not_null].str.replace(' ','_')

C:\Users\beto\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Indicators specified with API source in Helix

In [6]:
# point to indicators with helix: or helix code: in excel tmee data base
logic_helix = not_removed_indicators['Data Source'].str.lower().str.contains("helix:|helix code:|helix ")
# filter logic_helix that are not boolean
logic_helix_not_null = logic_helix.notnull()
# operator AND for two logics above
logic_helix_not_null = logic_helix & logic_helix_not_null
print(f"There are {logic_helix_not_null.sum()} indicators extracted from Helix.")

There are 28 indicators extracted from Helix.


In [7]:
# populate dataframe with the corresponding codes from Helix Source
# before: eliminate all Helix ... prefix
not_removed_indicators['Code'][logic_helix_not_null] = not_removed_indicators[logic_helix_not_null]['Data Source']\
.str.replace(r'^.*?: ', '', regex=True)\
.str.replace('/', '').str.replace('\\', '')
# consider something more elegant with regex for the line above?
# consider something stronger with respect to blank spaces in the regex line!

C:\Users\beto\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\beto\anaconda3\lib\site-packages\pandas\core\generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


#### Append info to Data Dictionary (Indicators Code), retrieved by API (from UIS & Helix)

In [13]:
# path to file for appending
path_write_file = './populate_data_dictionary/indicator_dictionary_TM_v2.xlsx'

In [ ]:
# use a function from stackoverflow to append data dictionary
append_df_to_excel(path_write_file, not_removed_indicators.iloc[21:,[105]], sheet_name='Indicator', startrow=22,
                   startcol = 5, header = False, index = False)

#### Write Source names for Helix & UIS

In [ ]:
# standardize helix Api source names adding 'Helix:' prefix to code names
not_removed_indicators['Source Name'] = 'Helix: ' + not_removed_indicators[logic_helix_not_null]['Code']

In [ ]:
# standardize UIS Api source names adding 'UIS:' prefix to code names
not_removed_indicators['Source Name'][logic_UIS_not_null] = 'UIS: ' + not_removed_indicators[logic_UIS_not_null]['Code']

#### Append info to Data Dictionary (Indicators Source Namce), retrieved by API (from UIS & Helix)

In [ ]:
# place list of helix and UIS api's in source
logic_helix_uis = not_removed_indicators['Source Name'].notna()
append_df_to_excel(path_write_file, not_removed_indicators.loc[logic_helix_uis,'Source Name'].iloc[2:],
                   sheet_name='Source', startrow=6,
                   startcol = 2, header = False, index = False)

#### Get Indicators Addresses, retrieved by UIS API

In [10]:
sources_uis = not_removed_indicators['Data Source'][logic_UIS_not_null]
# Keep from sources only the address 'root' before country specification
sources_uis = sources_uis.str.replace(r'^.*?https', 'https', regex=True).apply(lambda x: x.split('AL+')[0])
# note: the logic above doesn't work when indicator data source cell contains a new line before the address

#### Append info to Data Dictionary (Indicators Addresses), retrieved by UIS API

In [14]:
# place list UIS api's addresses in source spreadsheet
append_df_to_excel(path_write_file, sources_uis,
                   sheet_name='Source', startrow=26,
                   startcol = 3, header = False, index = False)

In [11]:
sources_uis

189    https://api.uis.unesco.org/sdmx/data/UNESCO,ED...
190    https://api.uis.unesco.org/sdmx/data/UNESCO,ED...
191    https://api.uis.unesco.org/sdmx/data/UNESCO,ED...
192    https://api.uis.unesco.org/sdmx/data/UNESCO,SD...
196    https://api.uis.unesco.org/sdmx/data/UNESCO,SD...
                             ...                        
387    https://api.uis.unesco.org/sdmx/data/UNESCO,SD...
388    https://api.uis.unesco.org/sdmx/data/UNESCO,SD...
389    https://api.uis.unesco.org/sdmx/data/UNESCO,SD...
390    https://api.uis.unesco.org/sdmx/data/UNESCO,SD...
391    https://api.uis.unesco.org/sdmx/data/UNESCO,SD...
Name: Data Source, Length: 129, dtype: object

In [ ]:
not_removed_indicators.loc[logic_helix_uis,'Source Name'].iloc[2:]

In [ ]:
not_removed_indicators.iloc[105]

In [ ]:
# create new column in dataframe with codes with add
# get indicator names

In [ ]:
# Keep from sources only the address 'root' before country specification
sources_uis[logic_api].str.replace(r'^.*?https', 'https', regex=True).apply(lambda x: x.split('AL+')[0])
# 'https://api.uis.unesco.org/sdmx/data/UNESCO,SDG4,2.0/PQTR..L3................'
# 'https://api.uis.unesco.org/sdmx/data/UNESCO,SDG4,2.0/PQTR..L3................'

In [ ]:
sources_uis = not_removed_indicators['Data Source'][logic_UIS_not_null]
logic_api = sources_uis.str.lower().str.contains("api.uis.unesco.org")

In [ ]:
# not_removed_indicators['Code'] = not_removed_indicators[logic_UIS_not_null]['Data Source']